## Grid explorer

In this notebook you will see how to:

- inspect the grid properties of GRIB data
- access the latitudes/longitudes of a field
- plot the gridpoints of a field
- interpolate GRIB data from one grid to another (regridding)
- extract the nearest gridpoint from a field

### Components of earthkit

This tutorial uses the following earthkit components - click any logo to open the package documentation:

<div align="center">
  <br>
  <a href="https://earthkit-data.readthedocs.io/en/latest/" target="_blank" style="display:inline-block; margin: 0 15px;">
    <img src="https://raw.githubusercontent.com/ecmwf/logos/refs/heads/main/logos/earthkit/earthkit-data-light.svg" alt="earthkit-data" width="200">
  </a>
  <a href="https://earthkit-plots.readthedocs.io/en/latest/" target="_blank" style="display:inline-block; margin: 0 15px;">
    <img src="https://raw.githubusercontent.com/ecmwf/logos/refs/heads/main/logos/earthkit/earthkit-plots-light.svg" alt="earthkit-plots" width="200">
  </a> 
  <a href="https://earthkit-regrid.readthedocs.io/en/latest/" target="_blank" style="display:inline-block; margin: 0 15px;">
    <img src="https://raw.githubusercontent.com/ecmwf/logos/refs/heads/main/logos/earthkit/earthkit-regrid-light.svg" alt="earthkit-regrid" width="200">
  </a>
  <a href="https://earthkit-geo.readthedocs.io/en/latest/" target="_blank" style="display:inline-block; margin: 0 15px;">
    <img src="https://raw.githubusercontent.com/ecmwf/logos/refs/heads/main/logos/earthkit/earthkit-geo-light.svg" alt="earthkit-geo" width="200">
  </a>
</div>

### 1. Getting the data

The input data is a GRIB file containing 1000 hPa temperature fields on 3 different global grids. 

First, fetch the file and list its contents.

In [1]:
import earthkit as ek

ds = ek.data.from_source("sample", "grids_3.grib")
ds.ls()

Adding method <property object at 0x30b5daf70> from <class 'earthkit.data.specs.time.Time'>.base_datetime
Adding method <property object at 0x30b5db010> from <class 'earthkit.data.specs.time.Time'>.valid_datetime
Adding method <property object at 0x30b5dbec0> from <class 'earthkit.data.specs.time.Time'>.step
Adding method <property object at 0x30b5dbfb0> from <class 'earthkit.data.specs.time.Time'>.forecast_reference_time
Adding method <property object at 0x30b600040> from <class 'earthkit.data.specs.time.Time'>.forecast_period
ALL_KEYS for <class 'earthkit.data.specs.field.time.TimeFieldSpec'>: ['base_datetime', 'valid_datetime', 'step', 'forecast_reference_time', 'forecast_period']


TypeError: argument of type 'NoneType' is not iterable

Next, select the field you will inspect in the rest of the notebook.

To try another grid type change the `gridType` in the `sel()` call below and rerun the notebook.

In [ ]:
# choose one from: regular_ll, reduced_gg, healpix
f = ds.sel(gridType="healpix")[0] 
f.metadata("gridType")

### 2. Inspecting the grid

With `dump()` you can inspect all the relevant GRIB metadata.

In [ ]:
f.dump(namespace="geography")

The field's shape is derived from the geography. For grids with  regular 2D structure, like regular latitude-longitude grids, the shape is always 2D. Otherwise, like for reduced Gaussian grids, it is 1D.

In [ ]:
f.shape

When you access the latitudes and longitudes via `to_latlon()` you get numpy arrays with the field's shape.

In [ ]:
ll = f.to_latlon()
ll["lat"].shape, ll["lon"].shape

In [ ]:
ll["lon"]

In [ ]:
ll["lat"]

You can access the field values either with `to_numpy()` or via the `values` property. By default, `to_numpy()` respects the field's shape unless you use the `flatten=True` option.

In [ ]:
f.to_numpy().shape, f.to_numpy(flatten=True).shape

When using the `values` property always a 1D array is returned.

In [ ]:
f.values.shape

### 3. Plotting the gridpoints

This example shows you how to plot the original gridpoint positions of the field.

In [ ]:
chart = ek.plots.Map(size=(7,7))
chart.contourf(f, units="celsius", auto_style=True)
# plot the original grid points
chart.grid_points(f, c="black") #marker="+"
chart.title(f"gridType={f.metadata('gridType')}")
chart.coastlines()
chart.gridlines()
chart.legend()

chart.show()

The next plot shows a smaller area and displays the grid values at each grid point.

In [ ]:
import cartopy.crs as ccrs

chart = ek.plots.Map(domain="Europe")
chart.contourf(f, units="celsius", auto_style=True)
# plot the original grid points
chart.grid_points(f, c="black") #marker="+"

# generate grid values
lat, lon, vals = f.data(flatten=True)
labels = [f"{x:.1f}" for x in vals]
for i, lbs in enumerate(labels):
    chart.ax.annotate(lbs, (lon[i], lat[i]), transform=ccrs.Geodetic(), 
                      xytext=(0,5),
                      textcoords="offset pixels", 
                      annotation_clip=True, 
                      fontsize=6, horizontalalignment="center")

chart.title(f"gridType={f.metadata('gridType')}")
chart.coastlines()
chart.gridlines()
chart.legend()

chart.show()

### 4. Regridding

You can regrid data with [earthkit.regrid.interpolate()](https://earthkit-regrid.readthedocs.io/en/latest/interpolate.html#interpolate). At the moment, earthkit-regrid uses a set of pre-generated interpolation weights downloaded automatically from a remote ECMWF repository. Only a limited set of global grid-to-grid combinations are supported (the ones the interpolation weights are available for). For the full list see the [inventory](https://earthkit-regrid.readthedocs.io/en/latest/inventory/index.html).

In [ ]:
# the target grid is a global 10x10 degree regular latitude-longitude grid
grid = {"grid": [10,10]}

# interpolate cannot read a single field but a fieldlist, so 
# we convert f into a fieldlist
ds_f =  ek.data.FieldList.from_fields([f])

# perform interpolation for each field and add results
# to a new fieldlist stored in memory
ds_res = ek.regrid.interpolate(ds_f, out_grid=grid, interpolation="linear")

The next cell plots the original and interpolated fields over a subarea.

In [ ]:
figure = ek.plots.Figure(domain="North Atlantic", rows=2, columns=1)

# the original field
subplot = figure.add_map(0, 0)
subplot.contourf(f, units="celsius", auto_style=True)
subplot.grid_points(f, c="black")
subplot.title("Original field")
# subplot.legend(label="")

# the interpolated field
subplot = figure.add_map(1, 0)
subplot.contourf(ds_res[0], units="celsius", auto_style=True)
subplot.grid_points(ds_res[0], c="black")
subplot.title("Interpolated field")
# subplot.legend(label="")

figure.coastlines()
figure.gridlines()

figure.show()

### 5. Getting the nearest gridpoint

In this example you can see how to extract the nearest gridpoint value of a field using [earthkit.geo.nearest_point_haversine()](https://earthkit-geo.readthedocs.io/en/latest/_api/geo/distance/index.html#geo.distance.nearest_point_haversine).

In [ ]:
# ref location (lat, lon)
p_ref = (51.45, -0.97)

# get latlon
latlon = f.to_latlon()
lat = latlon["lat"]
lon = latlon["lon"]

# get nearest point index
idx, dist = ek.geo.nearest_point_haversine(p_ref, (lat, lon))
idx

In [ ]:
# extract field value at given index
f.values[idx]